In [1]:
from langchain.chat_models import ChatCohere
from langchain.vectorstores import Chroma
from langchain.embeddings import CohereEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
import os
load_dotenv()
COHERE_API_KEY = os.getenv('COHERE_API_KEY')

In [2]:
embeddings = CohereEmbeddings(cohere_api_key=COHERE_API_KEY)
db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

In [3]:
chat = ChatCohere()

In [4]:
retriever = db.as_retriever()

In [5]:
template = """Fetch hotel names, hotel rating, address, attractions(if any), description, hotel facilities, 
map, phone number, pincode, website url below details based only on the following context,
if you don't know the answer just say I don't know, don't try to make up:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [6]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | chat
    | StrOutputParser()
)

In [7]:
print(chain.invoke("What are the amenities of Oyo 16360 Green Homes, Patna, Bihar?"))

Oyo 16360 Green Homes, Patna, Bihar offers the following amenities: 

- Free continental breakfast
- Free WiFi
- Daily smoke-free property
- Accessible bathroom (in select rooms)
- 24-hour front desk

Would you like to know more about this hotel?


In [8]:
print(chain.invoke("Suggest me hotels in Zanzibar City"))

I've found several hotels in Zanzibar City with details including hotel names, ratings, addresses, attractions, descriptions, facilities, and more. Here are the hotels that fit your criteria:

1. 27 Cafe Zanzibar Airport Hotel - Located 3.4 km from the food market, this hotel offers air-conditioned rooms, a lively cafe, and an on-site restaurant. It also provides a travel information office to help guests arrange safaris and flights.
2. Chuini Zanzibar Beach Lodge - This lodge features a private beach overlooking the bay and offers Zanzibari-style architecture with the remains of the historic Chuini Palace. Guests can enjoy international cuisine at the on-site restaurant or relax at the African-inspired spa.
3. Coral Reef Resort - This resort is situated in Pwani Mchangani, along the northeast coast of Zanzibar. It features a restaurant and an outdoor swimming pool with sun loungers, offering views of the Indian Ocean. The air-conditioned rooms come with a mosquito net, safe, and desk.

In [13]:
print(chain.invoke("Fetch the hotel booking website Dolphin Hostel"))

I found two hotels with the name "Dolphin Hotel":

- The first hotel is the Dolphin Hotel Apartments located in Dubai, United Arab Emirates. You can find information about this hotel on Booking.com by searching for "Dolphin Apartments" or by directly accessing the website `https://www.booking.com/hotel/ae/dolphin-apartments.html`. 

- The second hotel is the Dolphin Hotel located in Shah Alam, Malaysia. However, I could not find information on the booking website for this hotel. 

Would you like to know more about any of these hotels?
